In [1]:
%run QM9GNN2_Multitask.ipynb

Using TensorFlow backend.


Loading QM9 dataset.
Reading SDF


100%|██████████| 133885/133885 [01:22<00:00, 1623.48it/s]


In [2]:
import psi4
import time
from os import path, getcwd

In [3]:
def predict_property_neural(prop=None, mol_id=-1):
    if mol_id == -1:
        raise ValueError("ID must be between 1 and 133885")
    for cluster in clusters:
        if prop in cluster:
            model = create_hard_parameter_sharing_model(len(cluster))
            model.load_weights(generate_model_filename(cluster))
            model.compile(optimizer='adam', loss='mse')
            predictions = model.predict([[X_complete[mol_id-1]], [A_complete[mol_id-1]], [E_complete[mol_id-1]]])
            mean, std = 0, 1
            with open(generate_model_helper_filename(prop), 'r') as f:
                lines = f.readlines()
                mean = float(lines[0].strip())
                std = float(lines[1].strip())
            prediction = mean + std * predictions[cluster.index(prop)]
            return prediction[0][0]
    if prop == 'gap':
        lumo = predict_property_neural(prop='lumo', mol_id=mol_id)
        homo = predict_property_neural(prop='homo', mol_id=mol_id)
        return lumo - homo
    raise ValueError("Property was not found in clusters list")

In [4]:
def get_data_folder_path():
    return path.join(getcwd(), '..', '..', 'dsgdb9nsd')

In [5]:
def get_molecule_from_file(filenum):
    filepath = path.join(get_data_folder_path(), 
                           'dsgdb9nsd_' + str(filenum).zfill(6) + '.xyz')
    f = open(filepath, 'r')
    lines = f.readlines()
    f.close()
    num_atoms = int(lines[0])
    atom_list = lines[2:2+num_atoms]
    for i in range(len(atom_list)):
        atom_list[i] = atom_list[i][:atom_list[i].rfind("\t")] + "\n"
    return psi4.geometry("".join(atom_list))

In [6]:
def generate_dft_output_file_path(filenum):
    return path.join('psi4_output', 'output_'+str(filenum)+'.dat')

In [7]:
def process_molecule(filenum, thermochemical=False):
    psi4.core.set_output_file(generate_dft_output_file_path(filenum), False)
    psi4.set_memory('2 GB')
    molecule = get_molecule_from_file(filenum)
    if thermochemical:
        e, wfn = psi4.freq('b3lyp/cc-pvqz', molecule=molecule, return_wfn=True)
    else:
        e, wfn = psi4.energy('b3lyp/cc-pvqz', molecule=molecule, return_wfn=True)
    return wfn

In [8]:
def extract_rotational_constants(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find('Rotational constants:') > -1 and lines[i].find('[MHz]') > -1:
            words = lines[i].split()
            rot_constants = []
            for const in [words[4], words[7], words[10]]:     
                if const.isnumeric():
                    rot_constants.append(float(const)/1000)
                else:
                    rot_constants.append(const)
                    # rot_constants.append(float(const)/1000)
            return rot_constants
    return None, None, None

In [9]:
def extract_dipole_moment(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Dipole Moment: [D]") > -1:
            return lines[i+1][lines[i+1].find("Total:") + 6:]

In [10]:
def extract_homo_lumo(filenum, wfn):
    homo = wfn.epsilon_a_subset("AO", "ALL").get(wfn.nalpha())
    lumo = wfn.epsilon_a_subset("AO", "ALL").get(wfn.nalpha() + 1)
    return homo, lumo

In [11]:
def extract_gap(filenum, wfn):
    homo, lumo = extract_homo_lumo_gap(fileum, wfn)
    return lumo - homo

In [12]:
def extract_zpve(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total ZPE, Electronic energy at 0 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [13]:
def extract_zero_point_internal_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total E0, Electronic energy") > -1:
            words = lines[i].split()
            return words[-2]

In [14]:
def extract_internal_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total E, Electronic energy at  298.15 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [15]:
def extract_enthalpy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total H, Enthalpy at  298.15 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [16]:
def extract_gibbs_free_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total G,") > -1:
            words = lines[i].split()
            return words[-2]

In [17]:
def extract_cv(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find('Total Cv') > -1:
            words = lines[i].split()
            return words[2]

In [18]:
def batch_process(start_num, end_num, thermochemical=False):
    f = open("output.csv", "w")
    output_header = "Index,A,B,C,Dipole,HOMO,LUMO"
    if thermochemical:
        output_header += ",zpve,H 298.15,G 298.15"
    output_header += "\n"
    f.write(output_header)
    for filenum in range(start_num, end_num+1):
        wfn = process_molecule(filenum, thermochemical=thermochemical)
        a, b, c = extract_rotational_constants(filenum, wfn)
        dipole = extract_dipole_moment(filenum, wfn)
        homo, lumo = extract_homo_lumo(filenum, wfn)
        output = str(filenum) + "," + str(a) + "," + str(b) + "," + str(c) + "," + str(dipole) + "," + str(homo) + "," + str(lumo)
        if thermochemical:
            zpve = extract_zpve(filenum, wfn)
            enthalpy = extract_enthalpy(filenum, wfn)
            gibbs_free_energy = extract_gibbs_free_energy(filenum, wfn)
            output += "," + str(zpve) + "," + str(enthalpy) + "," + str(gibbs_free_energy)
        output += "\n" 
        f.write(output)
    f.close()

In [19]:
def predict_all_properties_dft(filenum=-1, thermochemical=False):
    if filenum == -1:
        raise ValueError("Filenum must be between 1 and 133885")
    wfn = process_molecule(filenum, thermochemical=thermochemical)
    a, b, c = extract_rotational_constants(filenum, wfn)
    dipole = extract_dipole_moment(filenum, wfn)
    homo, lumo = extract_homo_lumo(filenum, wfn)
    ret_dict = {'A': a, 'B': b, 'C': c, 'mu': dipole, 'homo': homo, 
                'lumo': lumo, 'gap': lumo-homo}
    if thermochemical:
        zpve = extract_zpve(filenum, wfn)
        internal_energy = extract_internal_energy(filenum, wfn)
        u0 = extract_zero_point_internal_energy(filenum, wfn)
        enthalpy = extract_enthalpy(filenum, wfn)
        gibbs_free_energy = extract_gibbs_free_energy(filenum, wfn)
        cv = extract_cv(filenum, wfn)
        ret_dict.update({'zpve': zpve, 'u0': u0, 'u298': internal_energy, 
                           'h298': enthalpy, 'g298': gibbs_free_energy, 'cv': cv})
    return ret_dict

In [20]:
def lookup_property(prop=None, mol_id=-1):
    return y_complete.loc[mol_id-1, prop]

In [21]:
def prompt_user_for_calculation():
    while True:
        num = -1
        while num < 1 or num > 133885:
            try:
                num = int(input('Choose a molecule index (1-133885): '))
            except ValueError:
                print("Please provide a valid number")
                num = -1
            
        properties = ['A', 'B', 'C', 'mu', 'homo', 'lumo', 'gap', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']
        prop = None
        while prop not in properties:
            print("Choose an available property from the following:")
            print("A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv")
            prop = input('Choose a property: ')
        
        dft = None
        while dft not in ['0', '1']:
            print("Choose whether to use DFT or neural methods")
            print("0 for neural methods, 1 for DFT")
            dft = input('Calculation type: ')
        
        if dft == '1':
            print('Beginning DFT calculation')
            start = time.time()
            thermochemical = prop in properties[7:]
            ret_dict = predict_all_properties_dft(num, thermochemical=thermochemical)
            print(ret_dict[prop])
            end = time.time()
            print('DFT calculation took', end-start, 's')
        else:
            print('Beginning neural calculation:')
            start = time.time()
            print(predict_property_neural(prop=prop, mol_id=num))
            end = time.time()
            print('Neural method took', end-start, 's')
        print('Actual data:')
        print(lookup_property(prop=prop, mol_id=num))

In [ ]:
prompt_user_for_calculation()

Choose a molecule index (1-133885): 5
Choose an available property from the following:
A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv
Choose a property: mu
Choose whether to use DFT or neural methods
0 for neural methods, 1 for DFT
Calculation type: 1
Beginning DFT calculation
     3.0189

DFT calculation took 3.986867666244507 s
Actual data:
2.8937
Choose a molecule index (1-133885): 5
Choose an available property from the following:
A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv
Choose a property: mu
Choose whether to use DFT or neural methods
0 for neural methods, 1 for DFT
Calculation type: 0
Beginning neural calculation:
3.9849966
Neural method took 9.03567910194397 s
Actual data:
2.8937


In [25]:
properties = ['A', 'B', 'C', 'mu', 'homo', 'lumo', 'gap', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']

for prop in properties:
    print(prop)
    errors = list()
    for index in np.random.choice(133885, 10):
        pred = predict_property_neural(prop=prop, mol_id=index+1)
        actual = lookup_property(prop=prop, mol_id=index+1)
        err = abs((pred-actual)/actual*100)
        print(err)
        errors.append(err)
    print('total err', sum(errors)/len(errors))

A
1.1613623179630275
20.504606959160597
7.558073157349356
17.206505545422598
6.062193201064268
19.51433782978936
30.86750877290762
2.9429857174754446
20.550303399482235
68.54530701702843
total err 19.491318391764292
B
5.034127253830896
13.461252012557937
42.45159417325174
5.189147901393879
2.086009478934388
32.586227415421035
42.7366950961002
26.059192520693426
21.32364147818258
9.674294009479265
total err 20.06021813398453
C
20.45554922033389
21.04882152342823
13.418838711642358
18.75813055465599
3.758312556059039
14.77832265846691
2.836188073717104
4.567384693968092
2.7798530248881255
21.34642059182666
total err 12.37478216089864
mu
51.18382252613235
31.83549277615066
14.108179638451093
18.11783918605934
0.29595150329160885
10.00537307850123
22.713979268316457
30.885189405860302
19.699462007898507
18.152045285333422
total err 21.699733467599497
homo
2.5577182805831176
0.13904429235396779
1.5258677442255824
3.3458760034144017
0.7691398705586275
6.063772160019287
4.01756512270987
2.342